In this notebook we will compare bnlearn's hillclimbing algorithm and tabu algorithm against Gobnilp. We'll do this with default settings and BIC scoring. First let's use bnlearn's hill-climbing algorithm to learn from its built-in alarm dataset.

In [1]:
library(bnlearn)
data(alarm)
alarmhc.bn <- hc(alarm)
alarmhc.bn
score(alarmhc.bn,alarm,type="bic")


Attaching package: ‘bnlearn’

The following object is masked from ‘package:stats’:

    sigma




  Bayesian network learned via Score-based methods

  model:
   [HIST][HRBP][PAP][FIO2][ANES][ERCA][LVF|HIST][PMB|PAP][ERLO|HRBP][PCWP|LVF]
   [HR|HRBP:ERLO][HREK|HR:ERCA][HRSA|HR:ERCA][LVV|PCWP:LVF][CCHL|HR][CVP|LVV]
   [MINV|CCHL][STKV|LVF:LVV][CO|STKV:HR][HYP|LVV:STKV][VALV|MINV][INT|MINV:VALV]
   [PVS|FIO2:VALV][ACO2|CCHL:VALV][PRSS|INT:VALV][SHNT|PMB:INT]
   [VLNG|MINV:INT:VALV][SAO2|SHNT:PVS][ECO2|ACO2:VLNG][KINK|PRSS:VLNG]
   [VTUB|PRSS:MINV:INT][TPR|SAO2:CCHL][DISC|VTUB][BP|TPR:CO][APL|TPR]
   [VMCH|DISC:VTUB][MVS|VMCH]
  nodes:                                 37 
  arcs:                                  53 
    undirected arcs:                     0 
    directed arcs:                       53 
  average markov blanket size:           3.46 
  average neighbourhood size:            2.86 
  average branching factor:              1.43 

  learning algorithm:                    Hill-Climbing 
  score:                                 BIC (disc.) 
  penalization coefficient:       

[1] -220761.7

OK, so hill-climbing has quickly found a BN with BIC score of -220761.687712649. Let's see how tabu search does.

In [2]:
alarmtabu.bn <- tabu(alarm)
alarmtabu.bn
score(alarmtabu.bn,alarm,type="bic")


  Bayesian network learned via Score-based methods

  model:
   [PAP][FIO2][LVF][ANES][ERLO][HR][ERCA][HIST|LVF][HRBP|ERLO:HR][HREK|HR:ERCA]
   [HRSA|HR:ERCA][PMB|PAP][LVV|LVF][CCHL|HR][CVP|LVV][PCWP|LVV][MINV|CCHL]
   [STKV|LVF:LVV][CO|STKV:HR][HYP|LVV:STKV][VALV|MINV][INT|MINV:VALV]
   [PVS|FIO2:VALV][ACO2|CCHL:VALV][PRSS|INT:VALV][SHNT|PMB:INT]
   [VLNG|MINV:INT:VALV][SAO2|SHNT:PVS][ECO2|ACO2:VLNG][KINK|PRSS:VLNG]
   [VTUB|PRSS:MINV:INT][TPR|SAO2:CCHL][DISC|VTUB][BP|TPR:CO][APL|TPR]
   [VMCH|DISC:VTUB][MVS|VMCH]
  nodes:                                 37 
  arcs:                                  51 
    undirected arcs:                     0 
    directed arcs:                       51 
  average markov blanket size:           3.41 
  average neighbourhood size:            2.76 
  average branching factor:              1.38 

  learning algorithm:                    Tabu Search 
  score:                                 BIC (disc.) 
  penalization coefficient:              4.951744

[1] -220727.3

Tabu search is also very quick and finds a slightly better network with a score of -220727.331509064.

Now let's see how Gobnilp does on the same data.

In [3]:
library(reticulate)
m <- import("pygobnilp.gobnilp")$Gobnilp()
m$learn(alarm,plot=FALSE,local_score_type='BIC')
m$learned_bn

**********
BN has score -218632.30999687716
**********
ACO2<-CCHL,VALV -7365.911141511497
CCHL<-SAO2,TPR -5079.249250888842
VALV<-INT,VLNG -3656.802744212182
ANES<- -6708.582939203498
APL<-TPR -920.8316095318736
TPR<- -21814.99513188178
BP<-CO,TPR -10757.751590048358
CO<-HR,STKV -5340.866593873166
SAO2<-PVS,SHNT -2469.044793071095
STKV<-HYP,LVF -9121.293128875895
HR<-CCHL -7361.101683937812
CVP<-LVV -6151.495470079892
LVV<-HYP,LVF -7523.033824999752
DISC<- -6447.082832428477
ECO2<-ACO2,VLNG -3540.0482171325107
VLNG<-INT,VTUB -7559.536030079051
ERCA<- -6480.204445349012
ERLO<- -3981.143384717203
FIO2<- -4007.5819752605926
HIST<- -4238.961962826263
HRBP<-ERLO,HR -2822.4332565574773
HREK<-ERCA,HR -3136.7392147761648
HRSA<-ERCA,HR -3236.13944426857
HYP<-APL -10075.035674773591
INT<- -6658.474123732306
KINK<-VLNG,VTUB -2857.4047588872477
VTUB<-DISC,VMCH -3418.0624644648697
LVF<-HIST -1195.1534680970262
MINV<-INT,VLNG -3909.1413764771555
MVS<- -8115.930179708216
PAP<-PMB -7991.586300445202
P

Gobnilp is a lot slower, but on the other hand, the learned BN has a much better BIC score of -218632.30999687716. Note that Gobnilp is being run here with its default limit of at most 3 parents for any node. It could be that an even higher-scoring network exists where some nodes have more than 3 parents.

We can get bnlearn to score the network learned by Gobnilp to check that we get the same BIC score.

In [4]:
score(model2network(m$learned_bn$bnlearn_modelstring()),alarm,type="bic")

[1] -218632.3

Sure enough we get the same score (modulo some numerical imprecision). Now let's run the same experiment with bnlearn's built-in hailfinder dataset

In [5]:
data(hailfinder)
hailfinderhc.bn <- hc(hailfinder)
hailfinderhc.bn
score(hailfinderhc.bn,hailfinder,type="bic")
hailfindertabu.bn <- tabu(hailfinder)
hailfindertabu.bn
score(hailfindertabu.bn,hailfinder,type="bic")
m$learn(hailfinder,plot=FALSE,local_score_type='BIC')
m$learned_bn
score(model2network(m$learned_bn$bnlearn_modelstring()),hailfinder,type="bic")


  Bayesian network learned via Score-based methods

  model:
   [N07muVerMo][SubjVertMo][QGVertMotion][SatContMoist][RaoContMoist]
   [VISCloudCov][IRCloudCover][AMInstabMt][WndHodograph][MorningBound]
   [LoLevMoistAd][Date][MorningCIN][LIfr12ZDENSd][AMDewptCalPl][LatestCIN][LLIW]
   [CombVerMo|N07muVerMo:SubjVertMo:QGVertMotion]
   [CombMoisture|SatContMoist:RaoContMoist][CombClouds|VISCloudCov:IRCloudCover]
   [Scenario|Date][CurPropConv|LatestCIN:LLIW][AreaMesoALS|CombVerMo]
   [AreaMoDryAir|CombVerMo:CombMoisture][ScenRelAMCIN|Scenario]
   [ScenRelAMIns|Scenario][ScenRel34|Scenario][ScnRelPlFcst|Scenario]
   [Dewpoints|Scenario][LowLLapse|Scenario][MeanRH|Scenario][MidLLapse|Scenario]
   [MvmtFeatures|Scenario][RHRatio|Scenario][SfcWndShfDis|Scenario]
   [SynForcng|Scenario][TempDis|Scenario][WindAloft|Scenario]
   [WindFieldMt|Scenario][WindFieldPln|Scenario]
   [CldShadeOth|CombVerMo:AreaMoDryAir:CombClouds]
   [AMCINInScen|ScenRelAMCIN:MorningCIN]
   [AMInsWliScen|ScenRelAMIns

[1] -990474.8


  Bayesian network learned via Score-based methods

  model:
   [N07muVerMo][SubjVertMo][QGVertMotion][SatContMoist][RaoContMoist]
   [VISCloudCov][IRCloudCover][AMInstabMt][WndHodograph][MorningBound]
   [LoLevMoistAd][Date][MorningCIN][LIfr12ZDENSd][AMDewptCalPl][LatestCIN][LLIW]
   [CombVerMo|N07muVerMo:SubjVertMo:QGVertMotion]
   [CombMoisture|SatContMoist:RaoContMoist][CombClouds|VISCloudCov:IRCloudCover]
   [Scenario|Date][CurPropConv|LatestCIN:LLIW][AreaMesoALS|CombVerMo]
   [AreaMoDryAir|CombVerMo:CombMoisture][ScenRelAMCIN|Scenario]
   [ScenRelAMIns|Scenario][ScenRel34|Scenario][ScnRelPlFcst|Scenario]
   [Dewpoints|Scenario][LowLLapse|Scenario][MeanRH|Scenario][MidLLapse|Scenario]
   [MvmtFeatures|Scenario][RHRatio|Scenario][SfcWndShfDis|Scenario]
   [SynForcng|Scenario][TempDis|Scenario][WindAloft|Scenario]
   [WindFieldMt|Scenario][WindFieldPln|Scenario]
   [CldShadeOth|CombVerMo:AreaMoDryAir:CombClouds]
   [AMCINInScen|ScenRelAMCIN:MorningCIN]
   [AMInsWliScen|ScenRelAMIns

[1] -990474.8

**********
BN has score -990162.80504007
**********
AMCINInScen<-MorningCIN,ScenRelAMCIN -14827.804016810533
ScenRelAMCIN<-ScnRelPlFcst -54.4691815389487
MorningCIN<- -22451.662191092444
AMDewptCalPl<- -21401.356146585404
AMInsWliScen<-AMDewptCalPl,ScenRelAMIns -19662.46130603437
ScenRelAMIns<-ScnRelPlFcst -272.3459076947435
AMInstabMt<- -21979.829675807607
AreaMesoALS<-CombVerMo -59.42092531521676
CombVerMo<-N07muVerMo,QGVertMotion,SubjVertMo -16140.954285953674
AreaMoDryAir<-AreaMesoALS,CombMoisture -15243.049284360615
CombMoisture<-RaoContMoist,SatContMoist -22201.12301415751
Boundaries<-MorningBound,OutflowFrMt,WndHodograph -15464.935101862524
OutflowFrMt<-InsInMt,WndHodograph -12684.806287718651
MorningBound<- -19880.990986381017
WndHodograph<- -27534.501610060397
CapChange<-CompPlFcst -29.71046265760838
CompPlFcst<-CldShadeConv,CldShadeOth,CombVerMo -20195.30081667199
CapInScen<-AMCINInScen,CapChange -8769.900372311302
CldShadeConv<-InsInMt,WndHodograph -11930.685819990164
InsInM

[1] -990162.8

Gobnilp takes a very long time to find a BN mainly due to this Python version of Gobnilp computing the necessary local scores slowly. The found BN has a BIC score of -990162.80504007 in contrast to -990474.753096428 BIC score achieved by hill-climbing and tabu.

Finally, let's have a look at the Insurance dataset

In [6]:
data(insurance)
insurancehc.bn <- hc(insurance)
insurancehc.bn
score(insurancehc.bn,insurance,type="bic")
insurancetabu.bn <- tabu(insurance)
insurancetabu.bn
score(insurancetabu.bn,insurance,type="bic")
m$learn(insurance,plot=FALSE,local_score_type='BIC')
m$learned_bn
score(model2network(m$learned_bn$bnlearn_modelstring()),insurance,type="bic")


  Bayesian network learned via Score-based methods

  model:
   [RuggedAuto][MakeModel|RuggedAuto][CarValue|RuggedAuto:MakeModel]
   [Mileage|CarValue][VehicleYear|MakeModel:Mileage:CarValue]
   [SocioEcon|VehicleYear:MakeModel][Antilock|VehicleYear:MakeModel]
   [Airbag|VehicleYear:MakeModel][ThisCarDam|Mileage:Antilock]
   [OtherCar|SocioEcon][Cushioning|RuggedAuto:Airbag]
   [Accident|ThisCarDam:RuggedAuto][ThisCarCost|ThisCarDam:CarValue]
   [MedCost|ThisCarDam:Cushioning][DrivQuality|Accident:Mileage]
   [Theft|ThisCarDam:ThisCarCost][OtherCarCost|RuggedAuto:Accident]
   [ILiCost|Accident][Age|SocioEcon:DrivQuality]
   [PropCost|ThisCarCost:OtherCarCost][GoodStudent|Age:SocioEcon]
   [SeniorTrain|Age:DrivQuality][RiskAversion|Age:DrivQuality:SeniorTrain]
   [DrivingSkill|RiskAversion:DrivQuality][HomeBase|SocioEcon:RiskAversion]
   [AntiTheft|SocioEcon:RiskAversion][DrivHist|RiskAversion:DrivingSkill]
  nodes:                                 27 
  arcs:                           

[1] -266113


  Bayesian network learned via Score-based methods

  model:
   [MakeModel][CarValue|MakeModel][Mileage|CarValue]
   [VehicleYear|MakeModel:Mileage:CarValue][SocioEcon|VehicleYear:MakeModel]
   [RuggedAuto|VehicleYear:MakeModel][Antilock|VehicleYear:MakeModel]
   [Airbag|VehicleYear:MakeModel][ThisCarDam|Mileage:Antilock]
   [OtherCar|SocioEcon][Cushioning|RuggedAuto:Airbag]
   [Accident|ThisCarDam:RuggedAuto][ThisCarCost|ThisCarDam:CarValue]
   [MedCost|ThisCarDam:Cushioning][DrivQuality|Accident:Mileage]
   [Theft|ThisCarDam:ThisCarCost][OtherCarCost|RuggedAuto:Accident]
   [ILiCost|Accident][Age|SocioEcon:DrivQuality]
   [PropCost|ThisCarCost:OtherCarCost][GoodStudent|Age:SocioEcon]
   [SeniorTrain|Age:DrivQuality][RiskAversion|Age:DrivQuality:SeniorTrain]
   [DrivingSkill|RiskAversion:DrivQuality][HomeBase|SocioEcon:RiskAversion]
   [AntiTheft|SocioEcon:RiskAversion][DrivHist|RiskAversion:DrivingSkill]
  nodes:                                 27 
  arcs:                           

[1] -265500.9

**********
BN has score -264708.85975077713
**********
Accident<-Antilock,DrivQuality -12310.195471829664
Antilock<-MakeModel,VehicleYear -3057.5537899833407
DrivQuality<-DrivingSkill,RiskAversion -4890.437103938427
Age<-RiskAversion,SocioEcon -17309.676907648834
SocioEcon<-MakeModel -15401.201288104912
RiskAversion<-SocioEcon -22043.400097873542
Airbag<-MakeModel,VehicleYear -4426.614280155776
MakeModel<- -25007.99981689388
VehicleYear<-SocioEcon -10055.937028683156
AntiTheft<-RiskAversion,SocioEcon -5427.903744351528
CarValue<-MakeModel,Mileage,VehicleYear -11811.232170180529
Mileage<-Accident,DrivQuality -23647.681978155444
Cushioning<-Airbag,RuggedAuto -14785.84850529706
RuggedAuto<-MakeModel,VehicleYear -12165.511166709071
DrivHist<-DrivingSkill,RiskAversion -9560.920896777803
DrivingSkill<-Age,SeniorTrain -17818.7405231424
SeniorTrain<-Age,RiskAversion -1712.9733331992325
GoodStudent<-Age,SocioEcon -1937.614966223779
HomeBase<-RiskAversion,SocioEcon -14317.997554087886
ILiCost<-A

[1] -264708.9

So hill-climbing finds a BN with BIC score -266113.028472967, tabu search manages -265500.923940094 and Gobnilp achieves -264708.85975077713. In this case Gobnilp was not too slow either (since Insurance only has 27 nodes and the default limit of at most 3 parents per node is in use.)